In [1]:
latest_gameweek = 34

In [2]:
import pandas as pd
import numpy as np
from pathlib import Path
import math

import catboost
#import shap

import plotly.graph_objects as go
import plotly.express as px
import seaborn as sns
import matplotlib.pyplot as plt
sns.set_style("darkgrid")

pd.set_option('display.max_columns', 300)
#pd.set_option('display.max_rows', 100)

In [3]:
# get login credentials for fetching team data
file_path = Path('../../login_credentials/fpl_login.json')
login_credentials = pd.read_json(file_path, typ='series')
# utility function for fetching team data
from src.utils import fetch_my_team

# Data

In [4]:
filepath = Path(f'../data/predictions/gameweek{latest_gameweek}.csv')
projections = pd.read_csv(filepath, index_col=0)
display(projections.head())
display(projections.shape)

,name,element_type,home,corners_and_indirect_freekicks_order,creativity_rank,direct_freekicks_order,ict_index_rank,influence_rank,minutes,now_cost,penalties_order,points_per_game,selected_by_percent,threat_rank,team_xG_ewm_5,team_xG_ewm_10,team_xG_ewm_20,team_xG_ewm_40,team_xGA_ewm_5,team_xGA_ewm_10,team_xGA_ewm_20,team_xGA_ewm_40,opponent_xG_ewm_5,opponent_xG_ewm_10,opponent_xG_ewm_20,opponent_xG_ewm_40,opponent_xGA_ewm_5,opponent_xGA_ewm_10,opponent_xGA_ewm_20,opponent_xGA_ewm_40,gameweek_assists_ewm_5,gameweek_bps_ewm_5,gameweek_creativity_ewm_5,event_points_ewm_5,gameweek_goals_scored_ewm_5,gameweek_goals_conceded_ewm_5,gameweek_saves_ewm_5,gameweek_threat_ewm_5,gameweek_xG_ewm_5,gameweek_xA_ewm_5,gameweek_xGA_ewm_5,gameweek_minutes_ewm_5,gameweek_xPoints_ewm_5,gameweek_assists_ewm_10,gameweek_bps_ewm_10,gameweek_creativity_ewm_10,event_points_ewm_10,gameweek_goals_scored_ewm_10,gameweek_goals_conceded_ewm_10,gameweek_saves_ewm_10,gameweek_threat_ewm_10,gameweek_xG_ewm_10,gameweek_xA_ewm_10,gameweek_xGA_ewm_10,gameweek_minutes_ewm_10,gameweek_xPoints_ewm_10,gameweek_assists_ewm_20,gameweek_bps_ewm_20,gameweek_creativity_ewm_20,event_points_ewm_20,gameweek_goals_scored_ewm_20,gameweek_goals_conceded_ewm_20,gameweek_saves_ewm_20,gameweek_threat_ewm_20,gameweek_xG_ewm_20,gameweek_xA_ewm_20,gameweek_xGA_ewm_20,gameweek_minutes_ewm_20,gameweek_xPoints_ewm_20,gameweek_assists_ewm_40,gameweek_bps_ewm_40,gameweek_creativity_ewm_40,event_points_ewm_40,gameweek_goals_scored_ewm_40,gameweek_goals_conceded_ewm_40,gameweek_saves_ewm_40,gameweek_threat_ewm_40,gameweek_xG_ewm_40,gameweek_xA_ewm_40,gameweek_xGA_ewm_40,gameweek_minutes_ewm_40,gameweek_xPoints_ewm_40,gameweek_assists_expanding,gameweek_bps_expanding,gameweek_creativity_expanding,event_points_expanding,gameweek_goals_scored_expanding,gameweek_goals_conceded_expanding,gameweek_saves_expanding,gameweek_threat_expanding,gameweek_xG_expanding,gameweek_xA_expanding,gameweek_xGA_expanding,gameweek_minutes_expanding,gameweek_xPoints_expanding,gameweek_assists_expanding_per90,gameweek_bps_expanding_per90,gameweek_creativity_expanding_per90,event_points_expanding_per90,gameweek_goals_scored_expanding_per90,gameweek_goals_conceded_expanding_per90,gameweek_saves_expanding_per90,gameweek_threat_expanding_per90,gameweek_xG_expanding_per90,gameweek_xA_expanding_per90,gameweek_xGA_expanding_per90,gameweek_xPoints_expanding_per90,xG_overperformance,team_name,opponent_team,date,gameweek,expected_points
0,Aaron Cresswell,2,1,3.0,338,NaN,446,449,321,42,NaN,0.9,0.3,620,1.354593,1.375486,1.367818,1.354578,2.069144,1.940926,1.825274,1.687124,2.327609,2.355220,2.271699,2.184424,1.111579,1.151261,1.179418,1.182833,0.031972,9.798750,9.744302,1.290207,0.008911,1.007822,0.0,0.565450,0.003478,0.050538,0.550104,46.157732,1.769273,0.092164,13.701845,14.429008,1.978417,0.024806,1.215073,0.0,1.514519,0.009667,0.072550,0.800170,57.513614,2.369443,0.148025,16.132181,17.385004,2.565292,0.033107,1.294323,0.0,2.094446,0.013926,0.091303,0.957803,65.710651,2.772282,0.183165,17.363156,18.903039,2.910866,0.035807,1.319343,0.0,2.337788,0.015950,0.103194,1.031455,69.963338,2.987950,6.0,501.0,550.2,95.0,1.0,36.0,0.0,68.0,0.5128,3.37947,31.7466,2146.0,86.502396,0.251631,21.011184,23.074557,3.984157,0.041938,1.509786,0.0,2.851817,0.021506,0.141730,1.331404,3.627780,1.950078,West Ham,Liverpool,2024-04-27T11:30:00Z,35,1.588187
1,Aaron Cresswell,2,0,3.0,338,NaN,446,449,321,42,NaN,0.9,0.3,620,1.354593,1.375486,1.367818,1.354578,2.069144,1.940926,1.825274,1.687124,2.154191,2.014599,1.875759,1.756910,1.518933,1.556748,1.518328,1.404261,0.031972,9.798750,9.744302,1.290207,0.008911,1.007822,0.0,0.565450,0.003478,0.050538,0.550104,46.157732,1.769273,0.092164,13.701845,14.429008,1.978417,0.024806,1.215073,0.0,1.514519,0.009667,0.072550,0.800170,57.513614,2.369443,0.148025,16.132181,17.385004,2.565292,0.033107,1.294323,0.0,2.094446,0.013926,0.091303,0.957803,65.710651,2.772282,0.183165,17.363156,18.903039,2.910866,0.035807,1.31934

(2460, 113)

In [5]:
filepath = Path('../data/fpl_df.csv')
fpl_df = pd.read_csv(filepath, index_col=0, low_memory=False)
fpl_df = fpl_df[fpl_df.season=='23-24']
display(fpl_df.head())
display(fpl_df.shape)

,assists,bonus,bps,clean_sheets,corners_and_indirect_freekicks_order,creativity,creativity_rank,creativity_rank_type,direct_freekicks_order,dreamteam_count,element_type,event_points,first_name,goals_conceded,goals_scored,ict_index,ict_index_rank,ict_index_rank_type,influence,influence_rank,influence_rank_type,minutes,now_cost,own_goals,penalties_missed,penalties_order,penalties_saved,points_per_game,red_cards,saves,second_name,selected_by_percent,threat,threat_rank,threat_rank_type,total_points,web_name,yellow_cards,team_name,gameweek,season,gameweek_xG,gameweek_xA,gameweek_xGA,gameweek_minutes,team_xG,team_xGA,team_xG_ewm_5,team_xG_ewm_10,team_xG_ewm_20,team_xG_ewm_40,team_xGA_ewm_5,team_xGA_ewm_10,team_xGA_ewm_20,team_xGA_ewm_40,opponent_xG,opponent_xGA,opponent_xG_ewm_5,opponent_xG_ewm_10,opponent_xG_ewm_20,opponent_xG_ewm_40,opponent_xGA_ewm_5,opponent_xGA_ewm_10,opponent_xGA_ewm_20,opponent_xGA_ewm_40,home,gameweek_assists,gameweek_bps,gameweek_creativity,gameweek_goals_scored,gameweek_goals_conceded,gameweek_own_goals,gameweek_penalties_saved,gameweek_red_cards,gameweek_saves,gameweek_threat,gameweek_yellow_cards,gameweek_xPoints,gameweek_assists_ewm_5,gameweek_bps_ewm_5,gameweek_creativity_ewm_5,event_points_ewm_5,gameweek_goals_scored_ewm_5,gameweek_goals_conceded_ewm_5,gameweek_saves_ewm_5,gameweek_threat_ewm_5,gameweek_xG_ewm_5,gameweek_xA_ewm_5,gameweek_xGA_ewm_5,gameweek_minutes_ewm_5,gameweek_xPoints_ewm_5,gameweek_assists_ewm_10,gameweek_bps_ewm_10,gameweek_creativity_ewm_10,event_points_ewm_10,gameweek_goals_scored_ewm_10,gameweek_goals_conceded_ewm_10,gameweek_saves_ewm_10,gameweek_threat_ewm_10,gameweek_xG_ewm_10,gameweek_xA_ewm_10,gameweek_xGA_ewm_10,gameweek_minutes_ewm_10,gameweek_xPoints_ewm_10,gameweek_assists_ewm_20,gameweek_bps_ewm_20,gameweek_creativity_ewm_20,event_points_ewm_20,gameweek_goals_scored_ewm_20,gameweek_goals_conceded_ewm_20,gameweek_saves_ewm_20,gameweek_threat_ewm_20,gameweek_xG_ewm_20,gameweek_xA_ewm_20,gameweek_xGA_ewm_20,gameweek_minutes_ewm_20,gameweek_xPoints_ewm_20,gameweek_assists_ewm_40,gameweek_bps_ewm_40,gameweek_creativity_ewm_40,event_points_ewm_40,gameweek_goals_scored_ewm_40,gameweek_goals_conceded_ewm_40,gameweek_saves_ewm_40,gameweek_threat_ewm_40,gameweek_xG_ewm_40,gameweek_xA_ewm_40,gameweek_xGA_ewm_40,gameweek_minutes_ewm_40,gameweek_xPoints_ewm_40,gameweek_assists_expanding,gameweek_bps_expanding,gameweek_creativity_expanding,event_points_expanding,gameweek_goals_scored_expanding,gameweek_goals_conceded_expanding,gameweek_saves_expanding,gameweek_threat_expanding,gameweek_xG_expanding,gameweek_xA_expanding,gameweek_xGA_expanding,gameweek_minutes_expanding,gameweek_xPoints_expanding,gameweek_assists_expanding_per90,gameweek_bps_expanding_per90,gameweek_creativity_expanding_per90,event_points_expanding_per90,gameweek_goals_scored_expanding_per90,gameweek_goals_conceded_expanding_per90,gameweek_saves_expanding_per90,gameweek_threat_expanding_per90,gameweek_xG_expanding_per90,gameweek_xA_expanding_per90,gameweek_xGA_expanding_per90,gameweek_minutes_expanding_per90,gameweek_xPoints_expanding_per90,xG_overperformance,chance_of_playing_next_round,chance_of_playing_this_round,code,cost_change_event,cost_change_event_fall,cost_change_start,cost_change_start_fall,ep_next,ep_this,form,id,in_dreamteam,news,news_added,photo,special,squad_number,status,team,team_code,transfers_in,transfers_in_event,transfers_out,transfers_out_event,value_form,value_season,starts,expected_goals,expected_assists,expected_goal_involvements,expected_goals_conceded,corners_and_indirect_freekicks_text,direct_freekicks_text,penalties_text,expected_goals_per_90,saves_per_90,expected_assists_per_90,expected_goal_involvements_per_90,expected_goals_conceded_per_90,goals_conceded_per_90,now_cost_rank,now_cost_rank_type,form_rank,form_rank_type,points_per_game_rank,points_per_game_rank_type,selected_rank,selected_rank_type,starts_per_90,clean_sheets_per_90,name,data_retrieved_datetime
18839,0,0,2,0,NaN,0.0,554

(9780, 209)

In [6]:
df = fpl_df.groupby('name').last().reset_index()[['id', 'name', 'team_name', 'element_type', 'now_cost', 
                            'gameweek_minutes_ewm_20', 'points_per_game', 'total_points', 
                            'gameweek_xPoints_ewm_5', 'gameweek_xPoints_ewm_10', 'gameweek_xPoints_ewm_20', 'gameweek_xPoints_ewm_40']]
df['games_played'] = np.round(np.where(df['points_per_game']!=0, df['total_points'] / df['points_per_game'], 0),0)
df['price'] = df['now_cost'] / 10.0
df['value'] = df['gameweek_xPoints_ewm_20'] / df['price']
df['value_points'] = np.sqrt( df['gameweek_xPoints_ewm_20'] *  df['value'])
df

,id,name,team_name,element_type,now_cost,gameweek_minutes_ewm_20,points_per_game,total_points,gameweek_xPoints_ewm_5,gameweek_xPoints_ewm_10,gameweek_xPoints_ewm_20,gameweek_xPoints_ewm_40,games_played,price,value,value_points
0,530.0,Aaron Cresswell,West Ham,2,42,65.710651,0.9,7,1.769273,2.369443,2.772282,2.987950,8.0,4.2,0.660067,1.352735
1,104.0,Aaron Hickey,Brentford,2,45,77.587197,1.9,17,2.526123,2.707357,2.819039,2.877821,9.0,4.5,0.626453,1.328908
2,17.0,Aaron Ramsdale,Arsenal,1,45,90.000000,3.3,20,3.789068,4.093285,4.399517,4.476464,6.0,4.5,0.977670,2.073952
3,675.0,Aaron Ramsey,Burnley,3,50,38.760392,1.3,18,1.644932,1.583594,1.570526,1.569442,14.0,5.0,0.314105,0.702361
4,401.0,Aaron Wan-Bissaka,Manchester Utd,2,44,82.225925,2.2,35,2.053557,2.478524,2.934714,3.247205,16.0,4.4,0.666980,1.399070
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
553,645.0,Youssef Ramalho Chermiti,Everton,4,48,4.922454,0.8,10,0.734728,0.874055,0.923552,0.942296,12.0,4.8,0.192407,0.421542
554,493.0,Yves Bissouma,Tottenham,3,50,74.903218,1.6,38,1.691171,1.874199,2.052937,2.185656,24.0,5.0,0.410587,0.918101
555,734.0,Zack Nelson,Luton,3,45,17.785428,1.0,2,1.619647,1.488610,1.477886,1.495103,2.0,4.5,0.328419,0.696682
556,594.0,Zeki Amdouni,Burnley,4,52,54.521238,2.5,74,1.386755,1.872864,2.161365,2.291115,30.0,5.2,0.415647,0.947821


In [7]:
# drop duplicate players (some players get new spelling for their name during the season causing duplicates)
duplicate_ids = df.loc[df.id.duplicated(), 'id'].unique()
for id in duplicate_ids:
    ix = df.loc[df.id==id, 'games_played'].idxmin()
    df = df.drop(ix)

In [8]:
expected_points_next_10gw = (projections[projections.gameweek.isin( np.arange(latest_gameweek+1, latest_gameweek+11, 1) )]
 .groupby('name')
 .sum()
 )[['expected_points']].reset_index().rename(columns={'expected_points':'expected_points_next_10_GW'})

expected_points_next_5gw = (projections[projections.gameweek.isin( np.arange(latest_gameweek+1, latest_gameweek+6, 1) )]
 .groupby('name')
 .sum()
 )[['expected_points']].reset_index().rename(columns={'expected_points':'expected_points_next_5_GW'})

expected_points_next_10gw

,name,expected_points_next_10_GW
0,Aaron Cresswell,7.272900
1,Aaron Hickey,9.102537
2,Aaron Ramsdale,15.206312
3,Aaron Ramsey,7.272721
4,Aaron Wan-Bissaka,12.048719
...,...,...
553,Youssef Ramalho Chermiti,5.395854
554,Yves Bissouma,11.863018
555,Zack Nelson,5.834730
556,Zeki Amdouni,8.328628


In [9]:
df = df.merge(expected_points_next_10gw, on='name', how='left')
df = df.merge(expected_points_next_5gw, on='name', how='left')
display(df.head())

,id,name,team_name,element_type,now_cost,gameweek_minutes_ewm_20,points_per_game,total_points,gameweek_xPoints_ewm_5,gameweek_xPoints_ewm_10,gameweek_xPoints_ewm_20,gameweek_xPoints_ewm_40,games_played,price,value,value_points,expected_points_next_10_GW,expected_points_next_5_GW
0,530.0,Aaron Cresswell,West Ham,2,42,65.710651,0.9,7,1.769273,2.369443,2.772282,2.987950,8.0,4.2,0.660067,1.352735,7.272900,7.272900
1,104.0,Aaron Hickey,Brentford,2,45,77.587197,1.9,17,2.526123,2.707357,2.819039,2.877821,9.0,4.5,0.626453,1.328908,9.102537,9.102537
2,17.0,Aaron Ramsdale,Arsenal,1,45,90.000000,3.3,20,3.789068,4.093285,4.399517,4.476464,6.0,4.5,0.977670,2.073952,15.206312,15.206312
3,675.0,Aaron Ramsey,Burnley,3,50,38.760392,1.3,18,1.644932,1.583594,1.570526,1.569442,14.0,5.0,0.314105,0.702361,7.272721,7.272721
4,401.0,Aaron Wan-Bissaka,Manchester Utd,2,44,82.225925,2.2,35,2.053557,2.478524,2.934714,3.247205,16.0,4.4,0.666980,1.399070,12.048719,12.048719


# Analysis

In [10]:
(df[(df.games_played>=5)]
        .sort_values(by='expected_points_next_10_GW', ascending=False)
        .head(20))[['name', 'team_name', 'price', 'games_played',
                    'points_per_game', 'total_points', 'gameweek_xPoints_ewm_20', 'expected_points_next_10_GW',
                    'value_points']]

,name,team_name,price,games_played,points_per_game,total_points,gameweek_xPoints_ewm_20,expected_points_next_10_GW,value_points
203,Heung-Min Son,Tottenham,9.9,29.0,6.4,186,4.876869,32.441576,1.549971
442,Phil Foden,Manchester City,8.2,30.0,6.1,183,5.507109,29.162474,1.923166
303,Kevin De Bruyne,Manchester City,10.4,12.0,5.2,62,5.126267,27.261376,1.589588
113,Cole Palmer,Chelsea,6.2,28.0,7.4,208,6.479756,26.592362,2.602333
168,Erling Haaland,Manchester City,14.3,26.0,6.7,174,6.529091,26.422248,1.726572
80,Brennan Johnson,Tottenham,5.8,29.0,3.8,110,3.700618,26.073622,1.536598
389,Mohamed Salah,Liverpool,13.6,27.0,6.9,187,6.614664,25.535598,1.793653
447,Raheem Sterling,Chelsea,6.9,27.0,4.0,107,3.681284,24.641069,1.401441
85,Bukayo Saka,Arsenal,9.0,31.0,6.3,196,5.801592,24.623297,1.933864
19,Alexander Isak,Newcastle Utd,8.0,24.0,5.5,132,5.431960,24.506999,1.920488


In [11]:
df[df['name'].str.contains('Pickford')][['name', 'team_name', 'price', 'games_played',
                    'points_per_game', 'total_points', 'gameweek_xPoints_ewm_5', 'gameweek_xPoints_ewm_10',
                    'gameweek_xPoints_ewm_20', 'expected_points_next_10_GW', 'value_points']]

,name,team_name,price,games_played,points_per_game,total_points,gameweek_xPoints_ewm_5,gameweek_xPoints_ewm_10,gameweek_xPoints_ewm_20,expected_points_next_10_GW,value_points
278,Jordan Pickford,Everton,4.7,33.0,3.7,121,3.908443,3.782141,3.742776,14.361281,1.726414


In [12]:
positions = ['GOALKEEPERS', 'DEFENDERS', 'MIDFIELDERS', 'FORWARDS']

for ix, position in enumerate(positions):
    print(position)
    display(
        (df[(df.games_played>=5) & (df.element_type==ix+1)]
        .sort_values(by='expected_points_next_10_GW', ascending=False) 
        .head(20))[['name', 'team_name', 'price', 'games_played',
                    'points_per_game', 'total_points', 'gameweek_xPoints_ewm_20',
                    'expected_points_next_10_GW', 'value_points']]
    )


GOALKEEPERS


,name,team_name,price,games_played,points_per_game,total_points,gameweek_xPoints_ewm_20,expected_points_next_10_GW,value_points
461,Robert Sánchez,Chelsea,4.7,16.0,3.7,59,4.061779,20.976073,1.873559
409,Nick Pope,Newcastle Utd,5.5,14.0,4.0,56,4.323629,19.358576,1.843602
156,Ederson Santana de Moraes,Manchester City,5.5,28.0,3.2,89,3.889337,19.110446,1.658419
554,Đorđe Petrović,Chelsea,4.6,16.0,2.9,46,3.274672,18.034319,1.526824
354,Martin Dubravka,Newcastle Utd,4.2,18.0,3.4,62,3.892259,17.578061,1.899228
191,Guglielmo Vicario,Tottenham,5.3,32.0,3.1,98,3.339271,17.244770,1.450487
495,Stefan Ortega Moreno,Manchester City,3.8,6.0,2.8,17,3.464676,16.542791,1.777341
136,David Raya Martin,Arsenal,5.2,27.0,4.3,115,4.714939,16.425288,2.067637
38,André Onana,Manchester Utd,5.0,32.0,3.6,114,4.383987,16.126824,1.960578
24,Alisson Ramses Becker,Liverpool,5.7,23.0,4.0,93,4.060613,15.471074,1.700804


DEFENDERS


,name,team_name,price,games_played,points_per_game,total_points,gameweek_xPoints_ewm_20,expected_points_next_10_GW,value_points
306,Kieran Trippier,Newcastle Utd,6.8,25.0,4.3,108,3.935968,23.112481,1.509375
439,Pedro Porro,Tottenham,5.9,29.0,3.8,110,3.892575,21.124055,1.602548
69,Benjamin White,Arsenal,6.0,32.0,4.6,147,4.525778,20.014987,1.847641
272,John Stones,Manchester City,5.3,15.0,3.5,52,3.910993,18.915729,1.698827
523,Trent Alexander-Arnold,Liverpool,8.4,23.0,4.9,113,4.766484,18.877166,1.644593
312,Kyle Walker,Manchester City,5.4,26.0,3.2,84,4.056030,18.610755,1.745437
541,William Saliba,Arsenal,5.9,33.0,4.2,140,4.809361,18.469682,1.979983
173,Fabian Schär,Newcastle Utd,5.6,32.0,3.8,120,3.577917,18.290913,1.511946
475,Rúben Gato Alves Dias,Manchester City,5.5,27.0,2.7,72,4.076816,18.146759,1.738360
400,Nathan Aké,Manchester City,5.1,24.0,3.5,83,4.115410,18.130174,1.822334


MIDFIELDERS


,name,team_name,price,games_played,points_per_game,total_points,gameweek_xPoints_ewm_20,expected_points_next_10_GW,value_points
203,Heung-Min Son,Tottenham,9.9,29.0,6.4,186,4.876869,32.441576,1.549971
442,Phil Foden,Manchester City,8.2,30.0,6.1,183,5.507109,29.162474,1.923166
303,Kevin De Bruyne,Manchester City,10.4,12.0,5.2,62,5.126267,27.261376,1.589588
113,Cole Palmer,Chelsea,6.2,28.0,7.4,208,6.479756,26.592362,2.602333
80,Brennan Johnson,Tottenham,5.8,29.0,3.8,110,3.700618,26.073622,1.536598
389,Mohamed Salah,Liverpool,13.6,27.0,6.9,187,6.614664,25.535598,1.793653
447,Raheem Sterling,Chelsea,6.9,27.0,4.0,107,3.681284,24.641069,1.401441
85,Bukayo Saka,Arsenal,9.0,31.0,6.3,196,5.801592,24.623297,1.933864
237,James Maddison,Tottenham,7.8,22.0,4.6,101,4.198269,24.319407,1.503221
355,Martin Ødegaard,Arsenal,8.6,30.0,5.2,157,4.818411,24.077891,1.643065


FORWARDS


,name,team_name,price,games_played,points_per_game,total_points,gameweek_xPoints_ewm_20,expected_points_next_10_GW,value_points
168,Erling Haaland,Manchester City,14.3,26.0,6.7,174,6.529091,26.422248,1.726572
19,Alexander Isak,Newcastle Utd,8.0,24.0,5.5,132,5.431960,24.506999,1.920488
410,Nicolas Jackson,Chelsea,6.8,28.0,3.8,107,4.125553,24.358657,1.582077
291,Julián Álvarez,Manchester City,6.4,31.0,4.5,139,4.714899,23.644163,1.863727
513,Timo Werner,Tottenham,6.3,12.0,3.8,45,4.148412,21.333244,1.652767
425,Ollie Watkins,Aston Villa,8.9,33.0,6.6,217,5.002420,20.370632,1.676815
286,João Pedro Junqueira de Jesus,Brighton,5.2,26.0,3.4,88,3.620965,17.532908,1.587898
110,Christopher Nkunku,Chelsea,7.3,7.0,2.7,19,2.560516,17.259113,0.947689
132,Darwin Núñez Ribeiro,Liverpool,7.8,31.0,4.1,127,4.085626,17.245430,1.462889
449,Rasmus Højlund,Manchester Utd,7.0,24.0,3.6,86,3.482622,16.887190,1.316307


In [13]:
display(
        (df[(df.games_played>=5) & (df.element_type==3) & (df.price<=6.6)]
        .sort_values(by='expected_points_next_10_GW', ascending=False) 
        .head(20))[['name', 'team_name', 'price', 'games_played',
                    'points_per_game', 'total_points', 'gameweek_xPoints_ewm_20',
                    'expected_points_next_10_GW', 'value_points']]
    )

,name,team_name,price,games_played,points_per_game,total_points,gameweek_xPoints_ewm_20,expected_points_next_10_GW,value_points
113,Cole Palmer,Chelsea,6.2,28.0,7.4,208,6.479756,26.592362,2.602333
80,Brennan Johnson,Tottenham,5.8,29.0,3.8,110,3.700618,26.073622,1.536598
435,Pascal Groß,Brighton,6.2,30.0,4.7,141,4.458642,22.372076,1.790634
45,Anthony Gordon,Newcastle Utd,6.1,30.0,5.3,158,3.998879,21.562495,1.619099
294,Jérémy Doku,Manchester City,6.4,23.0,3.7,85,3.372143,21.277203,1.332957
82,Bruno Guimarães Rodriguez Moura,Newcastle Utd,5.8,31.0,3.7,114,3.660624,20.606266,1.519991
116,Conor Gallagher,Chelsea,5.4,30.0,3.3,99,3.018689,19.554459,1.299037
465,Rodrigo Hernandez,Manchester City,5.6,28.0,4.6,129,3.788133,19.486533,1.600778
71,Bernardo Veiga de Carvalho e Silva,Manchester City,6.2,27.0,4.0,107,3.601918,19.113350,1.446565
314,Leandro Trossard,Arsenal,6.5,29.0,3.7,107,3.320422,19.036923,1.302377


In [14]:
fpl_df.loc[fpl_df['name'].str.contains('Gordon'), ['name', 'event_points', 'gameweek_xPoints', 'gameweek_xPoints_expanding_per90']] #André Onana

,name,event_points,gameweek_xPoints,gameweek_xPoints_expanding_per90
19039,Anthony Gordon,4,2.626723,3.943800
19338,Anthony Gordon,0,0.450016,3.637004
19628,Anthony Gordon,10,7.027497,4.216334
19934,Anthony Gordon,1,2.392712,3.993337
20244,Anthony Gordon,5,2.001202,3.776276
20532,Anthony Gordon,12,9.465485,4.392807
20840,Anthony Gordon,5,1.863483,4.163794
21464,Anthony Gordon,8,6.302418,4.427120
21773,Anthony Gordon,2,2.896936,4.307678
22068,Anthony Gordon,7,4.805883,4.343751


In [15]:
(projections[projections.gameweek==(latest_gameweek+1)]
 .groupby('name')
 .sum()
 .sort_values('expected_points', ascending=False).head(20))[['expected_points']]

,expected_points
name,
Heung-Min Son,10.464812
Cole Palmer,8.631596
Brennan Johnson,8.273245
Nicolas Jackson,7.876982
Raheem Sterling,7.874540
James Maddison,7.516690
Robert Sánchez,6.676541
Richarlison de Andrade,6.527328
Timo Werner,6.448425


In [16]:
(projections[(projections.gameweek.isin(np.arange(latest_gameweek+1, latest_gameweek+11)))]
 .groupby('name')
 .sum()
 .sort_values('expected_points', ascending=False).head(20))[['expected_points']]

,expected_points
name,
Heung-Min Son,32.441576
Phil Foden,29.162474
Kevin De Bruyne,27.261376
Cole Palmer,26.592362
Erling Haaland,26.422248
Brennan Johnson,26.073622
Mohamed Salah,25.535598
Raheem Sterling,24.641069
Bukayo Saka,24.623297


In [17]:
projections[projections['name'].str.contains('Onana')].name.unique()

array(['Amadou Onana', 'André Onana'], dtype=object)

In [18]:
fpl_df.loc[fpl_df['name'].str.contains('Haaland'), ['name', 'gameweek', 'gameweek_minutes', 'gameweek_xPoints', ]]

,name,gameweek,gameweek_minutes,gameweek_xPoints
19025,Erling Haaland,1,79.0,7.427418
19314,Erling Haaland,2,90.0,3.780063
19604,Erling Haaland,3,90.0,9.756795
19913,Erling Haaland,4,90.0,11.615016
20217,Erling Haaland,5,90.0,12.339916
20508,Erling Haaland,6,90.0,5.102241
20815,Erling Haaland,7,90.0,2.330523
21135,Erling Haaland,8,90.0,2.512229
21438,Erling Haaland,9,90.0,4.966760
21747,Erling Haaland,10,90.0,14.506034


# My team

In [19]:
my_team = fetch_my_team(login_credentials.user_name, login_credentials.password, login_credentials.team_id)

In [20]:
my_gameweek = latest_gameweek+1

my_starting_11_names = []
for pick in my_team['picks'][0:11]:
    my_starting_11_names.append( df.loc[df.id==pick['element'], 'name'].item() )

my_subs_names = []
for pick in my_team['picks'][11:]:
    my_subs_names.append( df.loc[df.id==pick['element'], 'name'].item() )

my_starting_11 = pd.DataFrame()
for my_name in my_starting_11_names:
    my_starting_11 = pd.concat([
                    my_starting_11, 
                    projections[(projections.gameweek==my_gameweek) & (projections.name.str.contains(my_name))]
                    ])

my_subs = pd.DataFrame()
for my_name in my_subs_names:
    my_subs = pd.concat([
                    my_subs, 
                    projections[(projections.gameweek==my_gameweek) & (projections.name.str.contains(my_name))]
                    ])

expected_points = my_starting_11['expected_points'].sum() + my_starting_11['expected_points'].max()
display(f'Expected points: {expected_points}')

display(my_starting_11.groupby('name').sum()[['element_type', 'home', 'expected_points']].sort_values('element_type'))
#display(my_starting_11.shape[0])

display(my_subs.groupby('name').sum()[['element_type', 'home', 'expected_points']])
#display(my_subs.shape[0])

'Expected points: 60.9097792000769'

,element_type,home,expected_points
name,,,
Jordan Pickford,1,1,3.222697
Andrew Robertson,2,0,4.317641
Benjamin White,2,0,4.438456
Gabriel dos Santos Magalhães,2,0,3.768720
Bukayo Saka,3,0,5.412634
Mohamed Salah,3,0,6.405788
Pablo Sarabia,3,1,4.498448
Darwin Núñez Ribeiro,4,0,4.407200
Dominic Solanke,4,1,4.239063


,element_type,home,expected_points
name,,,
Charlie Taylor,2,0,2.599478
Illia Zabarnyi,2,1,2.614145
Matt Turner,1,1,2.361848
Michael Olise,3,0,3.732469


In [ ]:
# my_gameweek = latest_gameweek+1

# my_starting_11_names = ['Pickford', 'Zabarn', 'Bradley', 'Konsa', 'Salah', 'Saka', 'Pascal Gro', 'Palmer', 'Solanke',  
#                   'Haaland', 'Watkins']
# my_subs_names = ['Turner', 'Sarabia', 'Taylor', 'Estupi',]

# my_starting_11 = pd.DataFrame()
# for my_name in my_starting_11_names:
#     my_starting_11 = pd.concat([
#                     my_starting_11, 
#                     projections[(projections.gameweek==my_gameweek) & (projections.name.str.contains(my_name))]
#                     ])

# my_subs = pd.DataFrame()
# for my_name in my_subs_names:
#     my_subs = pd.concat([
#                     my_subs, 
#                     projections[(projections.gameweek==my_gameweek) & (projections.name.str.contains(my_name))]
#                     ])

# expected_points = my_starting_11['expected_points'].sum() + my_starting_11['expected_points'].max()
# display(f'Expected points: {expected_points}')

# display(my_starting_11[['name', 'element_type', 'home', 'expected_points']])
# display(my_starting_11.shape[0])

# display(my_subs[['name', 'element_type', 'home', 'expected_points']])
# display(my_subs.shape[0])

In [ ]:
(projections[(projections.gameweek==(latest_gameweek+1))]
 .groupby('name')
 .sum()
 .sort_values('expected_points', ascending=False).head(20))[['now_cost', 'expected_points']]

In [ ]:
#players = ['Pervis Estupiñán', 'Kieran Trippier']
#colors = ['red', 'white']
players = ['Heung-Min Son', 'Bruno Borges Fernandes']
colors = ['white', 'red']

fig = go.Figure()
for count, player in enumerate(players):

    my_projections = projections[projections['name']==player].sort_values(by='gameweek')

    fig.add_trace(
        go.Scatter(
            x=np.unique(my_projections['gameweek']),
            y=my_projections.groupby('gameweek').sum()['expected_points'].cumsum(),
            mode="markers+lines",
            marker=dict(color=colors[count]),
            name=player + ' - projected',
            #hovertext=('Opponent: ' + my_projections['opp_team'] + 
            #           ', was_home: ' + my_projections['was_home'].astype(str)),
            showlegend=True,
            ),
    )

fig.update_layout(
    #title="",
    template='plotly_dark',
    xaxis_title="gameweek",
    yaxis_title='projected_points',
    #showlegend=True
)

In [ ]:
for player in players:
    my_projections = projections[projections['name']==player].sort_values(by='gameweek')
    print(player)
    display(my_projections[['opponent_team','home','gameweek', 'expected_points']].set_index('gameweek'))

In [ ]:
fpl_df[fpl_df['name'].str.contains('Trippier')].groupby('gameweek').sum()

In [ ]:
from scipy.ndimage import gaussian_filter

In [ ]:
mode = 'gameweek'
#mode = 'cumsum'

players = ['Phil Foden', 'Jarrod Bowen']
colors = ['red', 'blue']
#players = ['Heung-Min Son', 'Bruno Borges Fernandes']
#colors = ['white', 'red']

fig = go.Figure()
for count, player in enumerate(players):

    my_past_data = fpl_df[fpl_df['name']==player].sort_values(by='gameweek')
    my_projections = projections[projections['name']==player].sort_values(by='gameweek')
    
    x_past = np.unique(my_past_data['gameweek'])
    x_future = np.unique(my_projections['gameweek'])

    if mode=='gameweek':
        y_past = my_past_data.groupby('gameweek').sum()['gameweek_xPoints_ewm_5']
        y_future = my_projections.groupby('gameweek').sum()['expected_points']
    elif mode=='cumsum':
        y_past = my_past_data.groupby('gameweek').sum()['gameweek_xPoints'].cumsum()
        y_future = y_past.iloc[-1] + my_projections.groupby('gameweek').sum()['expected_points'].cumsum()
    else:
        print('Choose a mode for the graph!')

    fig.add_trace(
        go.Scatter(
            x=x_past,
            y=y_past,
            mode="markers+lines",
            marker=dict(color=colors[count]),
            name=player,            
            showlegend=True,
            ),
    )

    fig.add_trace(
        go.Scatter(
            x=x_future,
            y=y_future,
            mode="markers+lines",
            marker=dict(color=colors[count]),
            name=player,            
            showlegend=False,
            ),
    )

fig.update_layout(
    #title="",
    template='plotly_dark',
    xaxis_title="gameweek",
    yaxis_title='expected points',
    #showlegend=True
)

In [ ]:
players = ['Kieran Trippier', 'Trent Alexander-Arnold']

#my_fill = ['tozeroy', 'tonexty']
my_fill = ['tonexty', 'tonexty']
#players = ['Heung-Min Son', 'Bruno Borges Fernandes']
#colors = ['white', 'red']

fig = go.Figure()
for count, player in enumerate(players):

    my_past_data = fpl_df[fpl_df['name']==player].sort_values(by='gameweek')
    my_projections = projections[projections['name']==player].sort_values(by='gameweek')
    
    x_past = list(np.unique(my_past_data['gameweek']))
    x_future = list(np.unique(my_projections['gameweek']))
    my_x = x_past + x_future

    y_past = list(my_past_data.groupby('gameweek').sum()['gameweek_xPoints'])
    y_future = list(my_projections.groupby('gameweek').sum()['expected_points'])
    my_y = y_past + y_future
    my_y_filtered = gaussian_filter(y_past + y_future, sigma=2, mode='nearest')

    fig.add_trace(
        go.Scatter(
            x=my_x,
            y=my_y_filtered,
            mode="markers+lines",
            marker=dict(color=colors[count]),  
            fill=my_fill[count],    
            name=player,            
            showlegend=True,
            ),
    )

    if len(players)==1:
        fig.add_trace(
            go.Scatter(
                x=my_x,
                y=my_y,
                mode="markers",
                marker=dict(color='white'),
                name=player,            
                showlegend=False,
                ),
        )

fig.add_vline(x=latest_gameweek+0.5,)

fig.update_layout(
    #title="",
    template='plotly_dark',
    xaxis_title="gameweek",
    yaxis_title='expected points',
    #showlegend=True
)

In [ ]:
import math

In [ ]:
player = 'Cole Palmer'
my_past_data = fpl_df[fpl_df['name']==player].sort_values(by='gameweek')
y_past = list(my_past_data.groupby('gameweek').sum()['gameweek_xPoints'])
#min([math.floor(x) for x in y_past])
max([math.ceil(x) for x in y_past])

In [ ]:
player = 'Erling Haaland'
my_past_data = fpl_df[fpl_df['name']==player].sort_values(by='gameweek')
my_xpoints = list(my_past_data.groupby('gameweek').sum()['gameweek_xPoints'])
my_points = list(my_past_data.groupby('gameweek').sum()['event_points'])

bin_size = 1
x_start = min(min([math.floor(x) for x in my_xpoints]), min([math.floor(x) for x in my_points]))
x_end = max(max([math.ceil(x) for x in my_xpoints]), max([math.ceil(x) for x in my_points]))

fig = go.Figure()

fig.add_trace(
        go.Histogram(
            x=my_xpoints,   
            xbins=dict(start=x_start, end=x_end, size=bin_size),
            name=player,            
            showlegend=True,
            ),
    )

fig.add_trace(
        go.Histogram(
            x=my_points,   
            xbins=dict(start=x_start, end=x_end, size=bin_size),
            name=player,            
            showlegend=True,
            ),
    )

fig.update_layout(barmode='overlay')
fig.update_traces(opacity=0.75)

In [ ]:
players = ['Kieran Trippier', 'Trent Alexander-Arnold']
fpl_df[(fpl_df['name']==players[0]) | (fpl_df['name']==players[1])][['name','gameweek']]

In [ ]:
players = ['Kieran Trippier', 'Trent Alexander-Arnold']
fpl_df[(fpl_df['name']==players[0]) | (fpl_df['name']==players[1])].groupby(['name','gameweek']).sum()[['gameweek_xPoints', 'event_points']].reset_index()

In [ ]:
players = ['Kieran Trippier', 'Trent Alexander-Arnold']
mode = 'gameweek_xPoints' 
#mode = 'event_points'

if len(players)==1:
    my_past_data = (
        fpl_df[fpl_df['name']==players[0]]
        .groupby('gameweek')
        .sum()[['gameweek_xPoints', 'event_points', 'name']] 
    )   
elif len(players)==2:
    my_past_data = (
        fpl_df[(fpl_df['name']==players[0]) | (fpl_df['name']==players[1])]
        .groupby(['name','gameweek'])
        .sum()[['gameweek_xPoints', 'event_points']]
        .reset_index()
    )

sns.swarmplot(my_past_data, x=mode, orient='h', size=10, hue='name')
plt.xlabel(mode)
plt.show()

## SHAP

In [ ]:
y_past.iloc[-1]

In [ ]:
model_path = Path('../models/catboost_20240102-210118.cbm')
model = catboost.CatBoostRegressor()
model.load_model(model_path)

In [ ]:
features_no_shift = ['element_type', 'home', 'opponent_xG_ewm_5', 'opponent_xG_ewm_10',
       'opponent_xG_ewm_20', 'opponent_xG_ewm_40', 'opponent_xGA_ewm_5',
       'opponent_xGA_ewm_10', 'opponent_xGA_ewm_20',
       'opponent_xGA_ewm_40', ]

features_shift = ['corners_and_indirect_freekicks_order', 'creativity_rank', 
       'direct_freekicks_order', 'ict_index_rank', 'influence_rank',
       'minutes', 'now_cost', 'penalties_order', 'points_per_game', 
       'selected_by_percent', 'threat_rank',
       'team_xG_ewm_5', 'team_xG_ewm_10', 'team_xG_ewm_20',
       'team_xG_ewm_40', 'team_xGA_ewm_5', 'team_xGA_ewm_10',
       'team_xGA_ewm_20', 'team_xGA_ewm_40', 
       'gameweek_assists_ewm_5', 'gameweek_bps_ewm_5',
       'gameweek_creativity_ewm_5', 'event_points_ewm_5',
       'gameweek_goals_scored_ewm_5', 'gameweek_goals_conceded_ewm_5',
       'gameweek_saves_ewm_5', 'gameweek_threat_ewm_5',
       'gameweek_xG_ewm_5', 'gameweek_xA_ewm_5', 'gameweek_xGA_ewm_5',
       'gameweek_minutes_ewm_5', 'gameweek_xPoints_ewm_5',
       'gameweek_assists_ewm_10', 'gameweek_bps_ewm_10',
       'gameweek_creativity_ewm_10', 'event_points_ewm_10',
       'gameweek_goals_scored_ewm_10', 'gameweek_goals_conceded_ewm_10',
       'gameweek_saves_ewm_10', 'gameweek_threat_ewm_10',
       'gameweek_xG_ewm_10', 'gameweek_xA_ewm_10', 'gameweek_xGA_ewm_10',
       'gameweek_minutes_ewm_10', 'gameweek_xPoints_ewm_10',
       'gameweek_assists_ewm_20', 'gameweek_bps_ewm_20',
       'gameweek_creativity_ewm_20', 'event_points_ewm_20',
       'gameweek_goals_scored_ewm_20', 'gameweek_goals_conceded_ewm_20',
       'gameweek_saves_ewm_20', 'gameweek_threat_ewm_20',
       'gameweek_xG_ewm_20', 'gameweek_xA_ewm_20', 'gameweek_xGA_ewm_20',
       'gameweek_minutes_ewm_20', 'gameweek_xPoints_ewm_20',
       'gameweek_assists_ewm_40', 'gameweek_bps_ewm_40',
       'gameweek_creativity_ewm_40', 'event_points_ewm_40',
       'gameweek_goals_scored_ewm_40', 'gameweek_goals_conceded_ewm_40',
       'gameweek_saves_ewm_40', 'gameweek_threat_ewm_40',
       'gameweek_xG_ewm_40', 'gameweek_xA_ewm_40', 'gameweek_xGA_ewm_40',
       'gameweek_minutes_ewm_40', 'gameweek_xPoints_ewm_40',
       'gameweek_assists_expanding', 'gameweek_bps_expanding',
       'gameweek_creativity_expanding', 'event_points_expanding',
       'gameweek_goals_scored_expanding',
       'gameweek_goals_conceded_expanding', 'gameweek_saves_expanding',
       'gameweek_threat_expanding', 'gameweek_xG_expanding',
       'gameweek_xA_expanding', 'gameweek_xGA_expanding',
       'gameweek_minutes_expanding', 'gameweek_xPoints_expanding',
       'gameweek_assists_expanding_per90', 'gameweek_bps_expanding_per90',
       'gameweek_creativity_expanding_per90',
       'event_points_expanding_per90',
       'gameweek_goals_scored_expanding_per90',
       'gameweek_goals_conceded_expanding_per90',
       'gameweek_saves_expanding_per90',
       'gameweek_threat_expanding_per90', 'gameweek_xG_expanding_per90',
       'gameweek_xA_expanding_per90', 'gameweek_xGA_expanding_per90',
       'gameweek_xPoints_expanding_per90', 'xG_overperformance'
    ]

features = features_no_shift + features_shift

target = ['event_points']

In [ ]:
my_player = 'Konsa'
projections.loc[projections['name'].str.contains(my_player), ['name', 'team_name', 'home', 'expected_points']]

In [ ]:
explainer = shap.TreeExplainer(model)
shap_values = explainer(projections.loc[projections['name'].str.contains(my_player), features]);

In [ ]:
shap.plots.bar(shap_values, max_display=20)